#### IMF Data

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
response = requests.get('https://www.imf.org/external/datamapper/api/v1/NGDPD')
GDP_results = response.json()

GDP_results.get('values').get('NGDPD').get('USA')

In [ ]:
response = requests.get('https://www.imf.org/external/datamapper/api/v1/countries')
available_countries = response.json()

# Iterating over each key, value pair in the 'countries' dictionary
for c, details in available_countries.get('countries', {}).items():
    label = details.get('label') # Getting the value of 'label' key in the dictionary pointed by 'c'
    if label is not None:  # Check to ensure 'label' exists before printing
        print(label)

In [69]:
migration_df = pd.read_csv('IRS_county_to_county_migration_2017-2021.csv')

In [66]:
adj_df = migration_df[
    (migration_df['dest_county'] != 'Foreign - APO/FPO ZIPs') &
    (migration_df['dest_county'] != 'Other flows - South') &
    (migration_df['dest_county'] != 'Other flows - West') &
    (migration_df['dest_county'] != 'Other flows - Midwest') &
    (migration_df['dest_county'] != 'Other flows - Northeast')
]

adj_df

,year,dest_state_id,dest_county_id,orig_state_id,orig_county_id,dest_state,dest_county,returns,individuals,adj_gross_income
0,2017,1,1,96,0,AL,Autauga County Total Migration-US and Foreign,2400,5702,125069000.0
1,2017,1,1,97,0,AL,Autauga County Total Migration-US,2366,5573,122696000.0
2,2017,1,1,97,1,AL,Autauga County Total Migration-Same State,1516,3352,66187000.0
3,2017,1,1,97,3,AL,Autauga County Total Migration-Different State,850,2221,56510000.0
4,2017,1,1,98,0,AL,Autauga County Total Migration-Foreign,34,129,2373000.0
...,...,...,...,...,...,...,...,...,...,...
447612,2021,56,45,97,3,WY,Weston County Total Migration-Different State,79,158,4320000.0
447613,2021,56,45,56,45,WY,Weston County Non-migrants,2494,5159,167541000.0
447614,2021,56,45,56,5,WY,Campbell County,20,25,967000.0
447615,2021,56,45,58,0,SS,Other flows - Same State,59,82,1859000.0


In [68]:
adj_df.to_csv('IRS_county_to_county_migration_2017-2021.csv', index=False)